# HW03: ML Models w/ Embedding Layers

## Load Data

In [1]:
# Imports CSV to open raw data
import csv

# Imports numpy to read label data
import numpy as np

# Sets autocompletion
%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False

In [2]:
def load_data(prefix):
    """ Loads the data from raw (preprocessed) datasets.
    
    Args:
        prefix (str): string with path prefix to datasets.
    
    Returns:
        dict: dictionary with the splitted dataset.
    
    """
    
    # Initalizes the output dictionary
    output_dict = {
        "train":None,
        "test":None,
        "validation":None
    }
    
    # Intializes the suffix dictionary
    suffix_dict = {
        "train": ["X_train.csv", "y_train.csv"],
        "test": ["X_test.csv", "y_test.csv"],
        "validation": ["X_val.csv", "y_val.csv"],
    }
    
    # Iterates over 
    for dataset in output_dict.keys():
        
        # Retrieves suffixes from dictionaty
        x_suffix, y_suffix = suffix_dict[dataset]
        
        # Initializes list for data
        x_data = []
        
        # Loads X data into a list
        with open(prefix + x_suffix, 'r', encoding='latin-1') as data:
            for row in csv.reader(data):
                
                # Creates a base string
                sentence_string = ""
                
                # Iterates over subwords and appends to string
                for subword in row:
                    sentence_string += " "
                    sentence_string += subword
                
                # Appends the sentence string
                x_data.append([sentence_string])
        
        # Converts input data to numpy array
        x_data = np.asarray(x_data)
        
        # Loads labels
        y_data = np.loadtxt(prefix + y_suffix)
               
        # Stores data into dictionary
        output_dict[dataset] = [x_data, y_data]

    # Returns output file
    return output_dict

In [3]:
def store_dataset_files(data, path):
    """ Stores the data dictionaries into CSV files.
    
    Args:
        data (dict): data that will be converted.
        label_dict (dict): labels dictionary.
        path (str): prefix to dataset folder.
    
    """
    
    # Creates the vectors for the features
    x_train = np.squeeze(data["train"][0])
    x_val = np.squeeze(data["validation"][0])
    x_test = np.squeeze(data["test"][0])
    
    # Defines the function to create the label vector
    def one_hot_to_index(data):
        
        # Creates the output vector
        output = []
        
        # Iterates and store the class index
        for label in data:
            for i, val in enumerate(label):
                if val == 1:
                    output.append(i)
                    
        # Converts to numpy array and returns the vector
        return np.asarray(output)
    
    # Creates a vector with the class index
    y_train = one_hot_to_index(data["train"][1])
    y_val = one_hot_to_index(data["validation"][1])
    y_test = one_hot_to_index(data["test"][1])
    
    # Merges the features and data
    train_data = np.transpose(np.vstack((x_train, y_train)))
    val_data = np.transpose(np.vstack((x_val, y_val)))
    test_data = np.transpose(np.vstack((x_test, y_test)))
    
    # Stores the datasets
    np.savetxt(path + "train_ds.csv", train_data,
        delimiter=',', fmt="%s,%s", header="sentence, character",
        comments=""
    )
    
    np.savetxt(path + "val_ds.csv", val_data,
        delimiter=',', fmt="%s,%s", header="sentence, character",
        comments=""
    )
    
    np.savetxt(path + "test_ds.csv", test_data,
        delimiter=',', fmt="%s,%s", header="sentence, character",
        comments=""
    )
    

In [4]:
# Loads data as preprocessed
data = load_data("./data/simpsons/")

# Stores merged CSV files
store_dataset_files(data, "./data/simpsons/")

# Loads data as preprocessed
data = load_data("./data/friends/")

# Stores merged CSV files
store_dataset_files(data, "./data/friends/")

# Models definitions

In [5]:
# Imports the best ML library ever: Tensorflow
import tensorflow as tf

# Imports the Keras API for Tensorflow
from tensorflow import keras

# Imports the layers, optimizers and metrics from Keras
from tensorflow.keras import layers, optimizers, metrics

# Imports the callbacks submodule from Keras API
from tensorflow.keras import callbacks

In [6]:
def create_fcn_model(model_name, train_data, vocabulary_size, seq_len, embedding_dim, layers_def, output_dim):
    """ Creates a fully connected deep model.
    
    Args:
        model_name (str): name of the model.
        train_data (np.array): data to train the encoder layer.
        vocabulary_size (int): maximum size for vocabulary.
        seq_len (int): maximum size for sequence length.
        embeedding_dim (int): embedding dimensionality.
        layers_def (list): list with layers definition.
        output_dim (int): output dimension for model.
        
    Returns:
        (tf.keras.Sequential):
    
    """
    
    # Creates the encoder layer
    encoder = layers.experimental.preprocessing.TextVectorization(
        max_tokens=vocabulary_size,
        output_mode="int",
        output_sequence_length=seq_len
    )

    # Adapts the training dataset
    encoder.adapt(train_data)
    
    # Creates the root model
    model = keras.Sequential(name=model_name)
    
    # Adds an input layer
    model.add(keras.Input(shape=(1,), dtype=tf.string))
    
    # Adds the encoder layer
    model.add(encoder)
    
    # Adds the embedding layer
    model.add(layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=embedding_dim,
    ))

    # Flattens embedding data
    model.add(layers.GlobalAveragePooling1D())
    
    # Adds the hidden layers
    for n_hidden in layers_def:
        model.add(layers.Dense(n_hidden))
        model.add(layers.Dropout(0.2))
    
    # Adds the output layer
    model.add(layers.Dense(output_dim, activation="softmax"))
    
    # Returns the model
    return model


In [7]:
def create_recursive_model(
    model_name,
    dataset,
    vectorization,
    vocabulary_size,
    embedding_dim, 
    seq_len, 
    layers_def, 
    output_dim
):
    
    # Creates the encoder layer
    if vectorization == "int":
        encoder = layers.experimental.preprocessing.TextVectorization(
            max_tokens=vocabulary_size,
            output_mode=vectorization,
            output_sequence_length=seq_len
        )
        
        # Adapts the training dataset
        encoder.adapt(np.asarray(dataset["train"][0], dtype=np.str))
        
    else:
        encoder = layers.experimental.preprocessing.TextVectorization(
            max_tokens=vocabulary_size,
            output_mode=vectorization
        )
        
        # Adapts the training dataset
        encoder.adapt(np.asarray(dataset["train"][0], dtype=np.str))
    
    # Creates the root model
    model = keras.Sequential(name=model_name)
    
    # Adds an input layer
    model.add(keras.Input(shape=(1,), dtype=tf.string))

    # Adds the encoder layer
    model.add(encoder)
    
    # Adds the embedding layer
    model.add(layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=embedding_dim,
    ))
    
    # Creates a flag to add a flatten layer
    _added_flatten = False
    
    # Adds the hidden layers
    for n_hidden in layers_def:
        
        # Adds a LSTM layer
        if n_hidden[0] == "LSTM":
            if not _added_flatten:
                model.add(layers.LSTM(
                    n_hidden[1], 
                    return_sequences=n_hidden[2]
                ))
            else:
                raise Exception("Cannot add a recursive layer after a dense layer")
        
        # Adds a bidirectional LSTM layer
        elif n_hidden[0] == "BI_LSTM":
            if not _added_flatten:
                model.add(layers.Bidirectional(
                    layers.LSTM(
                        n_hidden[1],
                        return_sequences=n_hidden[2]
                )))
            else:
                raise Exception("Cannot add a recursive layer after a dense layer")
            
        # Adds a RNN layer
        elif n_hidden[0] == "RNN":
            if not _added_flatten:
                model.add(layers.SimpleRNN(
                    n_hidden[1], 
                    return_sequences=n_hidden[2]
                ))
            else:
                raise Exception("Cannot add a recursive layer after a dense layer")
                
        # Adds a GRU layer
        elif n_hidden[0] == "GRU":
            if not _added_flatten:
                model.add(layers.GRU(
                    n_hidden[1], 
                    return_sequences=n_hidden[2]
                ))
            else:
                raise Exception("Cannot add a recursive layer after a dense layer")
            
        # Adds a dense model
        else:
            
            # Adds a flatten layer if necessary
            if not _added_flatten:
                model.add(layers.Flatten())
                _added_flatten = True
            
            # Adds the dense layer and a dropout layer
            model.add(layers.Dense(n_hidden[1]))
            model.add(layers.Dropout(0.5))

    # Adds the output layer
    model.add(layers.Dense(output_dim, activation="softmax"))

    # Returns the model
    return model

In [8]:
def consolide_model(model,lr=1e-3):
    
    # Compiles the model
    model.compile(
        optimizer=optimizers.Adam(lr=lr),
        loss="categorical_crossentropy",
        metrics=[
            "accuracy",
            metrics.Precision(name="precision"),
            metrics.Recall(name="recall")
        ]
    )
    
    # Returns the compiled model
    return model
    

## Utilities

In [9]:
def create_results_directory():
    """ Creates the directories to store the results. """
    
    # Imports the OS library
    import os
    
    # Checks that the results directory exists
    if not os.path.isdir("./doc/data/em_models/"):
        os.mkdir("./doc/data/em_models/")


In [10]:
def create_results_file(name):
    
    # Imports required functions
    from os import remove
    from os.path import exists
    
    # Creates the full path for file
    path = "./doc/data/em_models/" + name + ".csv"
    
    # If file exists it is deleted
    if exists(path):
        remove(path)
        
    # Creates the file and writers the header row
    with open(path, "w+") as file:
        csv_file = csv.writer(file, delimiter=",")
        csv_file.writerow([
            "name",
            "model",
            'accuracy',
            'precision',
            'recall',
            "F1"
        ])
    

In [11]:
def write_results_on_file(name, values):
    
    # Defines the full path
    path = "./doc/data/em_models/" + name + ".csv"
    
    # Creates the results list 
    results = []
    
    # Modifies to stores floats with format
    for value in values:
        if type(value) == str:
            results.append(value)
        elif type(value) == float:
            results.append("{:5.4f}".format(value))
    
    # Creates the file and writes the results
    with open(path, "a") as file:
        
        # Creates the writer object
        _writer = csv.writer(file, delimiter=",")
        
        # Writes the values into file
        _writer.writerow(results)
        

In [12]:
def calculate_f1_score(precision, recall):
    numer = 2.0 * float(precision) * float(recall)
    denom = float(precision) + float(recall)
    try:
        return numer / denom
    except ZeroDivisionError as e:
        return 0

## Hyper-parameters settings

In [13]:
# This line prevents TF crashing when using convolutional networks
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

### Embedding dimensionality

In [57]:
# Sets task name
task_name = "embeddings"

# Sets a pool for embedding dimensionalities
embedding_list = [5, 10, 15, 20, 25, 50, 100, 150]

# Prefixes for dataset location
prefix_dict  = {
    "simpsons": "./data/simpsons/",
    "friends": "./data/friends/"
}

# Sequence length from preprocessing analysis
seq_len_dict = {
    "simpsons": 15,
    "friends": 19
}

# Output dimension dictionary
output_dict = {
    "simpsons": 4,
    "friends": 6
}

# Sets a vocabulary size variable
vocabulary_size = 10000

# Creates the results folder
create_results_directory()

# Creates the LR decrease callback
lr_decrease = callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    patience=1,
    cooldown=3,
    factor=0.1,
    verbose=1
)

# Creates a callback for early stopping
early_stopping = callbacks.EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=0
)

In [58]:
# Iterates over dataset prefixes
for dataset in prefix_dict.keys():
    
    # Defines the results filenames
    train_name = task_name + "_" + dataset + "_train"
    val_name = task_name + "_" + dataset + "_val"
    test_name = task_name + "_" + dataset + "_test"
    
    # Creates the results file
    create_results_file(train_name)
    create_results_file(val_name)
    create_results_file(test_name)
    
    # Gets the prefix
    prefix = prefix_dict[dataset]
    
    # Loads the data
    data = load_data(prefix)
    
    # Gets the sequences length
    seq_len = seq_len_dict[dataset]
    
    # Gets output dimension
    output_dim = output_dict[dataset]
    
    # Initializes an index for models
    model_index = 1
    
    # Iterates over different embedding dimensionalities
    for embedding in embedding_list:
        
        # Creates a name for the model
        name = dataset + "_" + str(embedding)
        
        # Prints information
        print("Training model: " + name)
        
        # Calculates the hidden layer size
        hidden_layer = int((embedding + output_dim) / 2)
        
        # Creates the model
        model = create_fcn_model(
            model_name = name, 
            train_data = data["train"][0],
            vocabulary_size=vocabulary_size,
            seq_len=seq_len,
            embedding_dim=embedding,
            layers_def=[hidden_layer],
            output_dim=output_dim
        )
        
        # Consolides the model
        model = consolide_model(model, lr=1e-4)
        
        # Trains the model
        history = model.fit(
            x=data["train"][0],
            y=data["train"][1],
            epochs=50,
            validation_data=(data["validation"][0], data["validation"][1]),
            callbacks=[lr_decrease, early_stopping],
            verbose=0
        )
                                       
        # Creates the training results
        train_results = [
            name,
            str(model_index),
            history.history["accuracy"][-1],
            history.history["precision"][-1],
            history.history["recall"][-1],
            calculate_f1_score(history.history["precision"][-1], history.history["recall"][-1])
        ]

        # Stores the training results
        write_results_on_file(train_name, train_results)
        
        # Creates the validation results
        val_results = [
            name,
            str(model_index),
            history.history["val_accuracy"][-1],
            history.history["val_precision"][-1],
            history.history["val_recall"][-1],
            calculate_f1_score(history.history["val_precision"][-1], history.history["val_recall"][-1])
        ]
        
        # Stores the validation results
        write_results_on_file(val_name, val_results)

        # Evaluates the model
        results = model.evaluate(
            x = data["test"][0],
            y = data["test"][1]
        )
        
        # Appends index and model to list
        results.insert(0, str(model_index))
        results.insert(0, name)
        
        # Writes results
        write_results_on_file(test_name, results)
        
        # Increments the model index
        model_index += 1
        

Training model: simpsons_5

Epoch 00028: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.

Epoch 00031: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-07.
304/304 [==============================] - 1s 2ms/step - loss: 1.1337 - accuracy: 0.5162 - precision: 0.6672 - recall: 0.2631
Training model: simpsons_10

Epoch 00023: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 2ms/step - loss: 1.1337 - accuracy: 0.5143 - precision: 0.6568 - recall: 0.2858
Training model: simpsons_15

Epoch 00018: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 2ms/step - loss: 1.1248 - accuracy: 0.5179 - precision: 0.6717 - recall: 0.2830
Training model: simpsons_20

Epoch 00018: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 2ms/step - loss: 1.1273 - accuracy: 0.5148 - precision: 0.65

### Sequence Length dimensionality

In [59]:
# Sets task name
task_name = "seq_len"

# Sets a pool for sequence lengths
seq_len_list = [5, 10, 15, 20, 25, 50, 100]

# Prefixes for dataset location
prefix_dict  = {
    "simpsons": "./data/simpsons/",
    "friends": "./data/friends/"
}

# Output dimension dictionary
output_dict = {
    "simpsons": 4,
    "friends": 6
}

# Sets a vocabulary size variable
vocabulary_size = 10000

# Sets the embedding size
embedding_size_dict = {
    "simpsons":150,
    "friends":150
}

# Creates the folders
create_results_directory()

# Creates the LR decrease callback
lr_decrease = callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    patience=1,
    cooldown=3,
    factor=0.1,
    verbose=1
)

# Creates a callback for early stopping
early_stopping = callbacks.EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=0
)

In [62]:
# Iterates over dataset prefixes
for _dataset in prefix_dict.keys():
    
    # Defines the results filenames
    train_name = task_name + "_" + _dataset + "_train"
    val_name = task_name + "_" + _dataset + "_val"
    test_name = task_name + "_" + _dataset + "_test"
    
    # Creates the results file
    create_results_file(train_name)
    create_results_file(val_name)
    create_results_file(test_name)
    
    # Initializes an index for models
    model_index = 1
    
    # Iterates over sequence length list
    for seq_len in seq_len_list:
    
        # Gets the prefix
        prefix = prefix_dict[_dataset]
    
        # Loads the data
        data = load_data(prefix)
    
        # Gets output dimension
        output_dim = output_dict[_dataset]
        
        # Creates a name for the model
        name = _dataset + "_" + str(seq_len)
        
        # Prints information
        print("Training model: " + name)
        
        # Sets the corresponding embedding size
        embedding_size = embedding_size_dict[_dataset]
        
        # Calculates the hidden layer size
        hidden_layer = int((embedding_size + output_dim) / 2)
        
        # Creates the model
        model = create_fcn_model(
            model_name = name, 
            train_data = data["train"][0],
            vocabulary_size=vocabulary_size,
            seq_len=seq_len,
            embedding_dim=embedding_size,
            layers_def=[hidden_layer],
            output_dim=output_dim
        )
        
        # Consolides the model
        model = consolide_model(model, lr=1e-4)
        
        # Trains the model
        history = model.fit(
            x=data["train"][0],
            y=data["train"][1],
            epochs=50,
            validation_data=(data["validation"][0], data["validation"][1]),
            callbacks=[lr_decrease, early_stopping],
            verbose=0
        )
        
        # Creates the training results
        train_results = [
            name,
            str(model_index),
            history.history["accuracy"][-1],
            history.history["precision"][-1],
            history.history["recall"][-1],
            calculate_f1_score(history.history["precision"][-1], history.history["recall"][-1])
        ]

        # Stores the training results
        write_results_on_file(train_name, train_results)
        
        # Creates the validation results
        val_results = [
            name,
            str(model_index),
            history.history["val_accuracy"][-1],
            history.history["val_precision"][-1],
            history.history["val_recall"][-1],
            calculate_f1_score(history.history["val_precision"][-1], history.history["val_recall"][-1])
        ]
        
        # Stores the validation results
        write_results_on_file(val_name, val_results)

        # Evaluates the model
        results = model.evaluate(
            x = data["test"][0],
            y = data["test"][1]
        )
        
        # Appends model to list
        results.insert(0, str(model_index))
        results.insert(0, name)
        
        # Writes results
        write_results_on_file(test_name, results)
        
        # Increments the model index
        model_index += 1
        

Training model: simpsons_5

Epoch 00005: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 2ms/step - loss: 1.1565 - accuracy: 0.4971 - precision: 0.6187 - recall: 0.2816
Training model: simpsons_10

Epoch 00005: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 2ms/step - loss: 1.1277 - accuracy: 0.5145 - precision: 0.6269 - recall: 0.3167
Training model: simpsons_15

Epoch 00006: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 3ms/step - loss: 1.1124 - accuracy: 0.5215 - precision: 0.6410 - recall: 0.3244
Training model: simpsons_20

Epoch 00007: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 3ms/step - loss: 1.1147 - accuracy: 0.5211 - precision: 0.6454 - recall: 0.3140
Training model: simpsons_25

Epoch 00007: ReduceLROnPlateau r

### Vocabulary Size

In [63]:
# Sets task name
task_name = "vocabulary_size"

# Sets a pool for sequence lengths
vocabulary_size_list = [1000, 2500, 5000, 10000, 15000]

# Prefixes for dataset location
prefix_dict  = {
    "simpsons": "./data/simpsons/",
    "friends": "./data/friends/"
}

# Output dimension dictionary
output_dict = {
    "simpsons": 4,
    "friends": 6
}

# Sets a vocabulary size variable
seq_len_dict = {
    "simpsons": 15,
    "friends": 20
}

# Sets the embedding size
embedding_dict = {
    "simpsons": 150,
    "friends": 150
}

# Creates the folders
create_results_directory()


# Creates the LR decrease callback
lr_decrease = callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    patience=1,
    cooldown=3,
    factor=0.1,
    verbose=1
)

# Creates a callback for early stopping
early_stopping = callbacks.EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=0
)

In [66]:
# Iterates over dataset prefixes
for _dataset in prefix_dict.keys():
    
    # Defines the results filenames
    train_name = task_name + "_" + _dataset + "_train"
    val_name = task_name + "_" + _dataset + "_val"
    test_name = task_name + "_" + _dataset + "_test"
    
    # Creates the results file
    create_results_file(train_name)
    create_results_file(val_name)
    create_results_file(test_name)
    
    # Initializes an index for models
    model_index = 1
    
    # Iterates over sequence length list
    for vocabulary_size in vocabulary_size_list:
    
        # Gets the prefix
        prefix = prefix_dict[_dataset]
    
        # Loads the data
        data = load_data(prefix)
    
        # Gets output dimension
        output_dim = output_dict[_dataset]
        
        # Creates a name for the model
        name = _dataset + "_" + str(vocabulary_size)
        
        # Prints information
        print("Training model: " + name)
        
        # Sets the sequence length for model
        seq_len = seq_len_dict[_dataset]
        
        # Sets the embedding size
        embedding_size = embedding_dict[_dataset]
        
        # Calculates the hidden layer size
        hidden_layer = int((embedding_size + output_dim) / 2)
        
        # Creates the model
        model = create_fcn_model(
            model_name = name, 
            train_data = data["train"][0],
            vocabulary_size=vocabulary_size,
            seq_len=seq_len,
            embedding_dim=embedding_size,
            layers_def=[hidden_layer],
            output_dim=output_dim
        )
        
        # Consolides the model
        model = consolide_model(model, lr=1e-4)
        
        # Trains the model
        history = model.fit(
            x=data["train"][0],
            y=data["train"][1],
            batch_size=25,
            epochs=50,
            validation_data=(data["validation"][0], data["validation"][1]),
            callbacks=[lr_decrease, early_stopping],
            verbose=0
        )
        
        # Creates the training results
        train_results = [
            name,
            str(model_index),
            history.history["accuracy"][-1],
            history.history["precision"][-1],
            history.history["recall"][-1],
            calculate_f1_score(history.history["precision"][-1], history.history["recall"][-1])
        ]

        # Stores the training results
        write_results_on_file(train_name, train_results)
        
        # Creates the validation results
        val_results = [
            name,
            str(model_index),
            history.history["val_accuracy"][-1],
            history.history["val_precision"][-1],
            history.history["val_recall"][-1],
            calculate_f1_score(history.history["val_precision"][-1], history.history["val_recall"][-1])
        ]
        
        # Stores the validation results
        write_results_on_file(val_name, val_results)

        # Evaluates the model
        results = model.evaluate(
            x = data["test"][0],
            y = data["test"][1]
        )
        
        # Appends model to list
        results.insert(0, str(model_index))
        results.insert(0, name)
        
        # Writes results
        write_results_on_file(test_name, results)
        
        # Increments the model index
        model_index += 1
        

Training model: simpsons_1000

Epoch 00007: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 3ms/step - loss: 1.1213 - accuracy: 0.5119 - precision: 0.6619 - recall: 0.2714
Training model: simpsons_2500

Epoch 00006: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 3ms/step - loss: 1.1137 - accuracy: 0.5208 - precision: 0.6485 - recall: 0.3042
Training model: simpsons_5000

Epoch 00007: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 3ms/step - loss: 1.1146 - accuracy: 0.5207 - precision: 0.6402 - recall: 0.3219
Training model: simpsons_10000

Epoch 00006: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 3ms/step - loss: 1.1126 - accuracy: 0.5218 - precision: 0.6348 - recall: 0.3385
Training model: simpsons_15000

Epoch 00006: Reduce

## FCN Models

In [30]:
# Sets task name
task_name = "fcn_models"

# Sets a pool for model definitions
model_list = [
    [25],
    [50],
    [75],
    [100],
    [125],
    [150],
    [25, 25],
    [50, 25],
    [50, 50],
    [75, 50],
    [100, 50],
    [150, 75],
    [75, 50, 25],
    [100, 50, 25],
    [150, 75, 25],
    [150, 150, 150]
]

# Prefixes for dataset location
prefix_dict  = {
    "simpsons": "./data/simpsons/",
    "friends": "./data/friends/"
}

# Output dimension dictionary
output_dict = {
    "simpsons": 4,
    "friends": 6
}

# Sets a vocabulary size variable
seq_len_dict = {
    "simpsons": 15,
    "friends": 20
}

# Sets the embedding size
embedding_dict = {
    "simpsons": 150,
    "friends": 150
}

vocabulary_size_dict = {
    "simpsons": 10000,
    "friends": 15000
}

# Creates the folders
create_results_directory()

# Creates the LR decrease callback
lr_decrease = callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    patience=1,
    cooldown=3,
    factor=0.1,
    verbose=1
)

# Creates a callback for early stopping
early_stopping = callbacks.EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=0
)

In [32]:
for dataset in prefix_dict.keys():
    
    # Defines the results filenames
    train_name = task_name + "_" + dataset + "_train"
    val_name = task_name + "_" + dataset + "_val"
    test_name = task_name + "_" + dataset + "_test"
    
    # Creates the results file
    create_results_file(train_name)
    create_results_file(val_name)
    create_results_file(test_name)
    
    # Initializes an index for models
    model_index = 1
    
    for model in model_list:
        
        # Gets the prefix
        prefix = prefix_dict[dataset]
    
        # Loads the data
        data = load_data(prefix)
    
        # Gets output dimension
        output_dim = output_dict[dataset]
        
        # Creates a name for the model
        name = dataset
        
        # Adds layers definition to model name
        for layer in model:
            name += "_" + str(layer)
            
        # Prints information
        print("Training model: " + name)
        
        # Sets the vocabulary size
        vocabulary_size = vocabulary_size_dict[dataset]
        
        # Sets the sequence length for model
        seq_len = seq_len_dict[dataset]
        
        # Sets the embedding size
        embedding_size = embedding_dict[dataset]
        
        # Creates the model
        model = create_fcn_model(
            model_name = name, 
            train_data = data["train"][0],
            vocabulary_size=vocabulary_size,
            seq_len=seq_len,
            embedding_dim=embedding_size,
            layers_def=model,
            output_dim=output_dim
        )
        
        # Consolides the model
        model = consolide_model(model, lr=1e-4)
        
        # Trains the model
        history = model.fit(
            x=data["train"][0],
            y=data["train"][1],
            epochs=50,
            validation_data=(data["validation"][0], data["validation"][1]),
            callbacks=[lr_decrease, early_stopping],
            verbose=0
        )

        # Creates the training results
        train_results = [
            name,
            str(model_index),
            history.history["accuracy"][-1],
            history.history["precision"][-1],
            history.history["recall"][-1],
            calculate_f1_score(history.history["precision"][-1], history.history["recall"][-1])
        ]

        # Stores the training results
        write_results_on_file(train_name, train_results)
        
        # Creates the validation results
        val_results = [
            name,
            str(model_index),
            history.history["val_accuracy"][-1],
            history.history["val_precision"][-1],
            history.history["val_recall"][-1],
            calculate_f1_score(history.history["val_precision"][-1], history.history["val_recall"][-1])
        ]
        
        # Stores the validation results
        write_results_on_file(val_name, val_results)

        # Evaluates the model
        results = model.evaluate(
            x = data["test"][0],
            y = data["test"][1]
        )
        
        # Appends model to list
        results.insert(0, str(model_index))
        results.insert(0, name)
        
        # Removes loss from results
        results.pop(2)
        
        # Adds F1 score to test
        results.append(calculate_f1_score(results[3], results[4]))
        
        # Writes results
        write_results_on_file(test_name, results)
        
        # Increments the model index
        model_index += 1
        

Training model: simpsons_25

Epoch 00008: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 2ms/step - loss: 1.1105 - accuracy: 0.5244 - precision: 0.6416 - recall: 0.3286
Training model: simpsons_50

Epoch 00006: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 3ms/step - loss: 1.1162 - accuracy: 0.5208 - precision: 0.6671 - recall: 0.2903
Training model: simpsons_75

Epoch 00006: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 3ms/step - loss: 1.1141 - accuracy: 0.5225 - precision: 0.6415 - recall: 0.3254
Training model: simpsons_100

Epoch 00006: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 3ms/step - loss: 1.1158 - accuracy: 0.5206 - precision: 0.6489 - recall: 0.3112
Training model: simpsons_125

Epoch 00005: ReduceLROnPlatea

### Simple RNN

In [33]:
# Sets task name
task_name = "simple_rnn"

# Sets a pool for model definitions
model_list = [
    [("RNN", 150, False), ("DENSE", 75)],
    [("RNN", 300, False), ("DENSE", 75)],
    [("RNN", 150, False), ("DENSE", 150), ("DENSE", 75), ("DENSE", 25)],
    [("RNN", 150, True), ("RNN", 150, False), ("DENSE", 150), ("DENSE", 75), ("DENSE", 25)],
    [("RNN", 300, True), ("RNN", 150, False), ("DENSE", 150), ("DENSE", 75), ("DENSE", 25)]
]

# Prefixes for dataset location
prefix_dict  = {
    "simpsons": "./data/simpsons/",
    "friends": "./data/friends/"
}

# Output dimension dictionary
output_dict = {
    "simpsons": 4,
    "friends": 6
}

# Sets a vocabulary size variable
seq_len_dict = {
    "simpsons": 15,
    "friends": 20
}

# Sets the embedding size
embedding_dict = {
    "simpsons": 150,
    "friends": 150
}

vocabulary_size_dict = {
    "simpsons": 10000,
    "friends": 15000
}

# Creates the folders
create_results_directory()

# Creates the LR decrease callback
lr_decrease = callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    patience=1,
    cooldown=3,
    factor=0.1,
    verbose=1
)

# Creates a callback for early stopping
early_stopping = callbacks.EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=0
)

In [34]:
for dataset in prefix_dict.keys():
    
    # Defines the results filenames
    train_name = task_name + "_" + dataset + "_train"
    val_name = task_name + "_" + dataset + "_val"
    test_name = task_name + "_" + dataset + "_test"
    
    # Creates the results file
    create_results_file(train_name)
    create_results_file(val_name)
    create_results_file(test_name)
    
    # Initializes an index for models
    model_index = 1
    
    for model in model_list:
        
        # Gets the prefix
        prefix = prefix_dict[dataset]
    
        # Loads the data
        data = load_data(prefix)
    
        # Gets output dimension
        output_dim = output_dict[dataset]
        
        # Creates a name for the model
        name = dataset
        
        # Adds layers definition to model name
        for layer in model:
            name += "_" + str(layer[0]) + str(layer[1])
            
        # Prints information
        print("Training model: " + name)
        
        # Sets the vocabulary size
        vocabulary_size = vocabulary_size_dict[dataset]
        
        # Sets the sequence length for model
        seq_len = seq_len_dict[dataset]
        
        # Sets the embedding size
        embedding_size = embedding_dict[dataset]
        
        # Creates the model
        model = create_recursive_model(
            model_name = name, 
            dataset= data,
            vectorization="int",
            vocabulary_size=vocabulary_size,
            seq_len=seq_len,
            embedding_dim=embedding_size,
            layers_def=model,
            output_dim=output_dim
        )
        
        # Consolides the model
        model = consolide_model(model, lr=1e-4)
        
        # Trains the model
        history = model.fit(
            x=data["train"][0],
            y=data["train"][1],
            batch_size=25,
            epochs=50,
            validation_data=(data["validation"][0], data["validation"][1]),
            callbacks=[lr_decrease, early_stopping],
            verbose=0
        )

        # Creates the training results
        train_results = [
            name,
            str(model_index),
            history.history["accuracy"][-1],
            history.history["precision"][-1],
            history.history["recall"][-1],
            calculate_f1_score(history.history["precision"][-1], history.history["recall"][-1])
        ]

        # Stores the training results
        write_results_on_file(train_name, train_results)
        
        # Creates the validation results
        val_results = [
            name,
            str(model_index),
            history.history["val_accuracy"][-1],
            history.history["val_precision"][-1],
            history.history["val_recall"][-1],
            calculate_f1_score(history.history["val_precision"][-1], history.history["val_recall"][-1])
        ]
        
        # Stores the validation results
        write_results_on_file(val_name, val_results)

        # Evaluates the model
        results = model.evaluate(
            x = data["test"][0],
            y = data["test"][1]
        )
        
        # Appends model to list
        results.insert(0, str(model_index))
        results.insert(0, name)
        
        # Removes loss from results
        results.pop(2)
        
        # Adds F1 score to test
        results.append(calculate_f1_score(results[3], results[4]))
        
        # Writes results
        write_results_on_file(test_name, results)
        
        # Increments the model index
        model_index += 1

Training model: simpsons_RNN150_DENSE75

Epoch 00003: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 3ms/step - loss: 1.1455 - accuracy: 0.4969 - precision: 0.6179 - recall: 0.2963
Training model: simpsons_RNN300_DENSE75

Epoch 00003: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 4ms/step - loss: 1.1492 - accuracy: 0.5029 - precision: 0.6070 - recall: 0.3325
Training model: simpsons_RNN150_DENSE150_DENSE75_DENSE25

Epoch 00003: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 4ms/step - loss: 1.1545 - accuracy: 0.4900 - precision: 0.6867 - recall: 0.2190
Training model: simpsons_RNN150_RNN150_DENSE150_DENSE75_DENSE25

Epoch 00004: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 5ms/step - loss: 1.1564 - accuracy: 0.5019 - precis

### Unidirectional LSTM

In [35]:
# Sets task name
task_name = "lstm"

# Sets a pool for model definitions
model_list = [
    [("LSTM", 150, False), ("DENSE", 75)],
    [("LSTM", 300, False), ("DENSE", 75)],
    [("LSTM", 150, False), ("DENSE", 150), ("DENSE", 75), ("DENSE", 25)],
    [("LSTM", 150, True), ("LSTM", 150, False), ("DENSE", 150), ("DENSE", 75), ("DENSE", 25)],
    [("LSTM", 300, True), ("LSTM", 150, False), ("DENSE", 150), ("DENSE", 75), ("DENSE", 25)]
]

# Prefixes for dataset location
prefix_dict  = {
    "simpsons": "./data/simpsons/",
    "friends": "./data/friends/"
}

# Output dimension dictionary
output_dict = {
    "simpsons": 4,
    "friends": 6
}

# Sets a vocabulary size variable
seq_len_dict = {
    "simpsons": 15,
    "friends": 20
}

# Sets the embedding size
embedding_dict = {
    "simpsons": 150,
    "friends": 150
}

vocabulary_size_dict = {
    "simpsons": 10000,
    "friends": 15000
}

# Creates the folders
create_results_directory()

# Creates the LR decrease callback
lr_decrease = callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    patience=1,
    cooldown=3,
    factor=0.1,
    verbose=1
)

# Creates a callback for early stopping
early_stopping = callbacks.EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=0
)

In [36]:
for dataset in prefix_dict.keys():
    
    # Defines the results filenames
    train_name = task_name + "_" + dataset + "_train"
    val_name = task_name + "_" + dataset + "_val"
    test_name = task_name + "_" + dataset + "_test"
    
    # Creates the results file
    create_results_file(train_name)
    create_results_file(val_name)
    create_results_file(test_name)
    
    # Initializes an index for models
    model_index = 1
    
    for model in model_list:
        
        # Gets the prefix
        prefix = prefix_dict[dataset]
    
        # Loads the data
        data = load_data(prefix)
    
        # Gets output dimension
        output_dim = output_dict[dataset]
        
        # Creates a name for the model
        name = dataset
        
        # Adds layers definition to model name
        for layer in model:
            name += "_" + str(layer[0]) + str(layer[1])
            
        # Prints information
        print("Training model: " + name)
        
        # Sets the vocabulary size
        vocabulary_size = vocabulary_size_dict[dataset]
        
        # Sets the sequence length for model
        seq_len = seq_len_dict[dataset]
        
        # Sets the embedding size
        embedding_size = embedding_dict[dataset]
        
        # Creates the model
        model = create_recursive_model(
            model_name = name, 
            dataset= data,
            vectorization="int",
            vocabulary_size=vocabulary_size,
            seq_len=seq_len,
            embedding_dim=embedding_size,
            layers_def=model,
            output_dim=output_dim
        )
        
        # Consolides the model
        model = consolide_model(model, lr=1e-4)
        
        # Trains the model
        history = model.fit(
            x=data["train"][0],
            y=data["train"][1],
            batch_size=25,
            epochs=50,
            validation_data=(data["validation"][0], data["validation"][1]),
            callbacks=[lr_decrease, early_stopping],
            verbose=0
        )
        
        # Creates the training results
        train_results = [
            name,
            str(model_index),
            history.history["accuracy"][-1],
            history.history["precision"][-1],
            history.history["recall"][-1],
            calculate_f1_score(history.history["precision"][-1], history.history["recall"][-1])
        ]

        # Stores the training results
        write_results_on_file(train_name, train_results)
        
        # Creates the validation results
        val_results = [
            name,
            str(model_index),
            history.history["val_accuracy"][-1],
            history.history["val_precision"][-1],
            history.history["val_recall"][-1],
            calculate_f1_score(history.history["val_precision"][-1], history.history["val_recall"][-1])
        ]
        
        # Stores the validation results
        write_results_on_file(val_name, val_results)

        # Evaluates the model
        results = model.evaluate(
            x = data["test"][0],
            y = data["test"][1]
        )
        
        # Appends model to list
        results.insert(0, str(model_index))
        results.insert(0, name)
        
        # Removes loss from results
        results.pop(2)
        
        # Adds F1 score to test
        results.append(calculate_f1_score(results[3], results[4]))
        
        # Writes results
        write_results_on_file(test_name, results)
        
        # Increments the model index
        model_index += 1

Training model: simpsons_LSTM150_DENSE75

Epoch 00004: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 3ms/step - loss: 1.1492 - accuracy: 0.5044 - precision: 0.6335 - recall: 0.3106
Training model: simpsons_LSTM300_DENSE75

Epoch 00003: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 3ms/step - loss: 1.1296 - accuracy: 0.5091 - precision: 0.6326 - recall: 0.3183
Training model: simpsons_LSTM150_DENSE150_DENSE75_DENSE25

Epoch 00004: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 3ms/step - loss: 1.1348 - accuracy: 0.5137 - precision: 0.6316 - recall: 0.3160
Training model: simpsons_LSTM150_LSTM150_DENSE150_DENSE75_DENSE25

Epoch 00003: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 4ms/step - loss: 1.1778 - accuracy: 0.4777 - p

### Bidirectional LSTM

In [37]:
# Sets task name
task_name = "bi_lstm"

# Sets a pool for model definitions
model_list = [
    [("BI_LSTM", 150, False), ("DENSE", 75)],
    [("BI_LSTM", 300, False), ("DENSE", 75)],
    [("BI_LSTM", 150, False), ("DENSE", 150), ("DENSE", 75), ("DENSE", 25)],
    [("BI_LSTM", 150, True), ("BI_LSTM", 150, False), ("DENSE", 150), ("DENSE", 75), ("DENSE", 25)],
    [("BI_LSTM", 300, True), ("BI_LSTM", 150, False), ("DENSE", 150), ("DENSE", 75), ("DENSE", 25)]
]

# Prefixes for dataset location
prefix_dict  = {
    "simpsons": "./data/simpsons/",
    "friends": "./data/friends/"
}

# Output dimension dictionary
output_dict = {
    "simpsons": 4,
    "friends": 6
}

# Sets a vocabulary size variable
seq_len_dict = {
    "simpsons": 15,
    "friends": 20
}

# Sets the embedding size
embedding_dict = {
    "simpsons": 150,
    "friends": 150
}

vocabulary_size_dict = {
    "simpsons": 10000,
    "friends": 15000
}

# Creates the folders
create_results_directory()

# Creates the LR decrease callback
lr_decrease = callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    patience=1,
    cooldown=3,
    factor=0.1,
    verbose=1
)

# Creates a callback for early stopping
early_stopping = callbacks.EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=0
)

In [38]:
for dataset in prefix_dict.keys():
    
    # Defines the results filenames
    train_name = task_name + "_" + dataset + "_train"
    val_name = task_name + "_" + dataset + "_val"
    test_name = task_name + "_" + dataset + "_test"
    
    # Creates the results file
    create_results_file(train_name)
    create_results_file(val_name)
    create_results_file(test_name)
    
    # Initializes an index for models
    model_index = 1
    
    for model in model_list:
        
        # Gets the prefix
        prefix = prefix_dict[dataset]
    
        # Loads the data
        data = load_data(prefix)
    
        # Gets output dimension
        output_dim = output_dict[dataset]
        
        # Creates a name for the model
        name = dataset
        
        # Adds layers definition to model name
        for layer in model:
            name += "_" + str(layer[0]) + str(layer[1])
            
        # Prints information
        print("Training model: " + name)
        
        # Sets the vocabulary size
        vocabulary_size = vocabulary_size_dict[dataset]
        
        # Sets the sequence length for model
        seq_len = seq_len_dict[dataset]
        
        # Sets the embedding size
        embedding_size = embedding_dict[dataset]
        
        # Creates the model
        model = create_recursive_model(
            model_name = name, 
            dataset= data,
            vectorization="int",
            vocabulary_size=vocabulary_size,
            seq_len=seq_len,
            embedding_dim=embedding_size,
            layers_def=model,
            output_dim=output_dim
        )
        
        # Consolides the model
        model = consolide_model(model, lr=1e-4)
        
        # Trains the model
        history = model.fit(
            x=data["train"][0],
            y=data["train"][1],
            batch_size=25,
            epochs=50,
            validation_data=(data["validation"][0], data["validation"][1]),
            callbacks=[lr_decrease, early_stopping],
            verbose=0
        )
        
        # Creates the training results
        train_results = [
            name,
            str(model_index),
            history.history["accuracy"][-1],
            history.history["precision"][-1],
            history.history["recall"][-1],
            calculate_f1_score(history.history["precision"][-1], history.history["recall"][-1])
        ]

        # Stores the training results
        write_results_on_file(train_name, train_results)
        
        # Creates the validation results
        val_results = [
            name,
            str(model_index),
            history.history["val_accuracy"][-1],
            history.history["val_precision"][-1],
            history.history["val_recall"][-1],
            calculate_f1_score(history.history["val_precision"][-1], history.history["val_recall"][-1])
        ]
        
        # Stores the validation results
        write_results_on_file(val_name, val_results)

        # Evaluates the model
        results = model.evaluate(
            x = data["test"][0],
            y = data["test"][1]
        )
        
        # Appends model to list
        results.insert(0, str(model_index))
        results.insert(0, name)
        
        # Removes loss from results
        results.pop(2)
        
        # Adds F1 score to test
        results.append(calculate_f1_score(results[3], results[4]))
        
        # Writes results
        write_results_on_file(test_name, results)
        
        # Increments the model index
        model_index += 1

Training model: simpsons_BI_LSTM150_DENSE75

Epoch 00003: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 4ms/step - loss: 1.1170 - accuracy: 0.5244 - precision: 0.6286 - recall: 0.3424
Training model: simpsons_BI_LSTM300_DENSE75

Epoch 00003: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 4ms/step - loss: 1.1131 - accuracy: 0.5186 - precision: 0.6613 - recall: 0.2944
Training model: simpsons_BI_LSTM150_DENSE150_DENSE75_DENSE25

Epoch 00004: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 4ms/step - loss: 1.1337 - accuracy: 0.5008 - precision: 0.6516 - recall: 0.2734
Training model: simpsons_BI_LSTM150_BI_LSTM150_DENSE150_DENSE75_DENSE25

Epoch 00003: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 5ms/step - loss: 1.1294 - accur

### GRU

In [39]:
# Sets task name
task_name = "gru"

# Sets a pool for model definitions
model_list = [
    [("GRU", 150, False), ("DENSE", 75)],
    [("GRU", 300, False), ("DENSE", 75)],
    [("GRU", 150, False), ("DENSE", 150), ("DENSE", 75), ("DENSE", 25)],
    [("GRU", 150, True), ("GRU", 150, False), ("DENSE", 150), ("DENSE", 75), ("DENSE", 25)],
    [("GRU", 300, True), ("GRU", 150, False), ("DENSE", 150), ("DENSE", 75), ("DENSE", 25)]
]

# Prefixes for dataset location
prefix_dict  = {
    "simpsons": "./data/simpsons/",
    "friends": "./data/friends/"
}

# Output dimension dictionary
output_dict = {
    "simpsons": 4,
    "friends": 6
}

# Sets a vocabulary size variable
seq_len_dict = {
    "simpsons": 15,
    "friends": 20
}

# Sets the embedding size
embedding_dict = {
    "simpsons": 150,
    "friends": 150
}

vocabulary_size_dict = {
    "simpsons": 10000,
    "friends": 15000
}

# Creates the folders
create_results_directory()

# Creates the LR decrease callback
lr_decrease = callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    patience=1,
    cooldown=3,
    factor=0.1,
    verbose=1
)

# Creates a callback for early stopping
early_stopping = callbacks.EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=0
)

In [40]:
for dataset in prefix_dict.keys():
    
    # Defines the results filenames
    train_name = task_name + "_" + dataset + "_train"
    val_name = task_name + "_" + dataset + "_val"
    test_name = task_name + "_" + dataset + "_test"
    
    # Creates the results file
    create_results_file(train_name)
    create_results_file(val_name)
    create_results_file(test_name)
    
    # Initializes an index for models
    model_index = 1
    
    for model in model_list:
        
        # Gets the prefix
        prefix = prefix_dict[dataset]
    
        # Loads the data
        data = load_data(prefix)
    
        # Gets output dimension
        output_dim = output_dict[dataset]
        
        # Creates a name for the model
        name = dataset
        
        # Adds layers definition to model name
        for layer in model:
            name += "_" + str(layer[0]) + str(layer[1])
            
        # Prints information
        print("Training model: " + name)
        
        # Sets the vocabulary size
        vocabulary_size = vocabulary_size_dict[dataset]
        
        # Sets the sequence length for model
        seq_len = seq_len_dict[dataset]
        
        # Sets the embedding size
        embedding_size = embedding_dict[dataset]
        
        # Creates the model
        model = create_recursive_model(
            model_name = name, 
            dataset= data,
            vectorization="int",
            vocabulary_size=vocabulary_size,
            seq_len=seq_len,
            embedding_dim=embedding_size,
            layers_def=model,
            output_dim=output_dim
        )
        
        # Consolides the model
        model = consolide_model(model, lr=1e-4)
        
        # Trains the model
        history = model.fit(
            x=data["train"][0],
            y=data["train"][1],
            batch_size=25,
            epochs=50,
            validation_data=(data["validation"][0], data["validation"][1]),
            callbacks=[lr_decrease, early_stopping],
            verbose=0
        )
        
        # Creates the training results
        train_results = [
            name,
            str(model_index),
            history.history["accuracy"][-1],
            history.history["precision"][-1],
            history.history["recall"][-1],
            calculate_f1_score(history.history["precision"][-1], history.history["recall"][-1])
        ]

        # Stores the training results
        write_results_on_file(train_name, train_results)
        
        # Creates the validation results
        val_results = [
            name,
            str(model_index),
            history.history["val_accuracy"][-1],
            history.history["val_precision"][-1],
            history.history["val_recall"][-1],
            calculate_f1_score(history.history["val_precision"][-1], history.history["val_recall"][-1])
        ]
        
        # Stores the validation results
        write_results_on_file(val_name, val_results)

        # Evaluates the model
        results = model.evaluate(
            x = data["test"][0],
            y = data["test"][1]
        )
        
        # Appends model to list
        results.insert(0, str(model_index))
        results.insert(0, name)
        
        # Removes loss from results
        results.pop(2)
        
        # Adds F1 score to test
        results.append(calculate_f1_score(results[3], results[4]))
        
        # Writes results
        write_results_on_file(test_name, results)
        
        # Increments the model index
        model_index += 1

Training model: simpsons_GRU150_DENSE75

Epoch 00005: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 4ms/step - loss: 1.1482 - accuracy: 0.5119 - precision: 0.6253 - recall: 0.3200
Training model: simpsons_GRU300_DENSE75

Epoch 00005: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 5ms/step - loss: 1.1627 - accuracy: 0.5035 - precision: 0.6089 - recall: 0.3229
Training model: simpsons_GRU150_DENSE150_DENSE75_DENSE25

Epoch 00006: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 4ms/step - loss: 1.1660 - accuracy: 0.5038 - precision: 0.6343 - recall: 0.2901
Training model: simpsons_GRU150_GRU150_DENSE150_DENSE75_DENSE25

Epoch 00005: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 2s 5ms/step - loss: 1.1502 - accuracy: 0.5065 - precis

### Improved Search on Best Model

In [41]:
# Sets task name
task_name = "bi_lstm_improved"

# Sets a pool for model definitions
model_list = [
    [("BI_LSTM", 25, False), ("DENSE", 15)],
    [("BI_LSTM", 50, False), ("DENSE", 27)],
    [("BI_LSTM", 75, False), ("DENSE", 40)],
    [("BI_LSTM", 150, False), ("DENSE", 77)],
    [("BI_LSTM", 300, False), ("DENSE", 152)],
]

# Prefixes for dataset location
prefix_dict  = {
    "simpsons": "./data/simpsons/",
    "friends": "./data/friends/"
}

# Output dimension dictionary
output_dict = {
    "simpsons": 4,
    "friends": 6
}

# Sets a vocabulary size variable
seq_len_dict = {
    "simpsons": 15,
    "friends": 20
}

# Sets the embedding size
embedding_dict = {
    "simpsons": 150,
    "friends": 150
}

vocabulary_size_dict = {
    "simpsons": 10000,
    "friends": 15000
}

# Creates the folders
create_results_directory()

# Creates the LR decrease callback
lr_decrease = callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    patience=1,
    cooldown=3,
    factor=0.1,
    verbose=1
)

# Creates a callback for early stopping
early_stopping = callbacks.EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=0
)

In [42]:
for dataset in prefix_dict.keys():
    
    # Defines the results filenames
    train_name = task_name + "_" + dataset + "_train"
    val_name = task_name + "_" + dataset + "_val"
    test_name = task_name + "_" + dataset + "_test"
    
    # Creates the results file
    create_results_file(train_name)
    create_results_file(val_name)
    create_results_file(test_name)
    
    # Initializes an index for models
    model_index = 1
    
    for model in model_list:
        
        # Gets the prefix
        prefix = prefix_dict[dataset]
    
        # Loads the data
        data = load_data(prefix)
    
        # Gets output dimension
        output_dim = output_dict[dataset]
        
        # Creates a name for the model
        name = dataset
        
        # Adds layers definition to model name
        for layer in model:
            name += "_" + str(layer[0]) + str(layer[1])
            
        # Prints information
        print("Training model: " + name)
        
        # Sets the vocabulary size
        vocabulary_size = vocabulary_size_dict[dataset]
        
        # Sets the sequence length for model
        seq_len = seq_len_dict[dataset]
        
        # Sets the embedding size
        embedding_size = embedding_dict[dataset]
        
        # Creates the model
        model = create_recursive_model(
            model_name = name, 
            dataset= data,
            vectorization="int",
            vocabulary_size=vocabulary_size,
            seq_len=seq_len,
            embedding_dim=embedding_size,
            layers_def=model,
            output_dim=output_dim
        )
        
        # Consolides the model
        model = consolide_model(model, lr=1e-4)
        
        # Trains the model
        history = model.fit(
            x=data["train"][0],
            y=data["train"][1],
            batch_size=25,
            epochs=50,
            validation_data=(data["validation"][0], data["validation"][1]),
            callbacks=[lr_decrease, early_stopping],
            verbose=0
        )
        
        # Creates the training results
        train_results = [
            name,
            str(model_index),
            history.history["accuracy"][-1],
            history.history["precision"][-1],
            history.history["recall"][-1],
            calculate_f1_score(history.history["precision"][-1], history.history["recall"][-1])
        ]

        # Stores the training results
        write_results_on_file(train_name, train_results)
        
        # Creates the validation results
        val_results = [
            name,
            str(model_index),
            history.history["val_accuracy"][-1],
            history.history["val_precision"][-1],
            history.history["val_recall"][-1],
            calculate_f1_score(history.history["val_precision"][-1], history.history["val_recall"][-1])
        ]
        
        # Stores the validation results
        write_results_on_file(val_name, val_results)

        # Evaluates the model
        results = model.evaluate(
            x = data["test"][0],
            y = data["test"][1]
        )
        
        # Appends model to list
        results.insert(0, str(model_index))
        results.insert(0, name)
        
        # Removes loss from results
        results.pop(2)
        
        # Adds F1 score to test
        results.append(calculate_f1_score(results[3], results[4]))
        
        # Writes results
        write_results_on_file(test_name, results)
        
        # Increments the model index
        model_index += 1

Training model: simpsons_BI_LSTM25_DENSE15

Epoch 00005: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 2s 6ms/step - loss: 1.1203 - accuracy: 0.5210 - precision: 0.6304 - recall: 0.3253
Training model: simpsons_BI_LSTM50_DENSE27

Epoch 00004: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 1s 5ms/step - loss: 1.1187 - accuracy: 0.5147 - precision: 0.6557 - recall: 0.3054
Training model: simpsons_BI_LSTM75_DENSE40

Epoch 00004: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 2s 5ms/step - loss: 1.1271 - accuracy: 0.5247 - precision: 0.6280 - recall: 0.3325
Training model: simpsons_BI_LSTM150_DENSE77

Epoch 00003: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 2s 6ms/step - loss: 1.1226 - accuracy: 0.5091 - precision: 0.6591 - recall: 0.2788

### Vectorizations

In [50]:
train_ds = tf.data.experimental.make_csv_dataset(
    "./data/simpsons/train_ds.csv",
    batch_size=32,
    column_names = ["sentence", "character"],
    label_name="character",
    num_epochs=1
)

val_ds = tf.data.experimental.make_csv_dataset(
    "./data/simpsons/val_ds.csv",
    batch_size=32,
    column_names = ["sentence", "character"],
    label_name="character",
    num_epochs=1
)

test_ds = tf.data.experimental.make_csv_dataset(
    "./data/simpsons/test_ds.csv",
    batch_size=32,
    column_names = ["sentence", "character"],
    label_name="character",
    num_epochs=1
)

# Sets the vocabulary size and sequence length
VOCAB = 10000
MAX_TOKENS=15

# Sets the vectorization
VECT = "binary"

# Creates the LR decrease callback
lr_decrease = callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    patience=1,
    cooldown=3,
    factor=0.1,
    verbose=1
)

# Creates a callback for early stopping
early_stopping = callbacks.EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=0
)

In [51]:
# Sets the task name
task_name = "binary_vectorization"

name=task_name

# Defines the results filenames
train_name = task_name + "_" + "simpsons_train"
val_name = task_name + "_" + "simpsons_val"
test_name = task_name + "_" + "simpsons_test"

# Creates the results file
create_results_file(train_name)
create_results_file(val_name)
create_results_file(test_name)

# Maps to create train text dataset
train_text = train_ds.map(lambda x, y: x["sentence"])

# Creates the vectorization layer
bin_vectorize_layer = layers.experimental.preprocessing.TextVectorization(
    output_mode=VECT,
    max_tokens=VOCAB,
    pad_to_max_tokens=MAX_TOKENS
)

# Trains the vectorization layer
bin_vectorize_layer.adapt(train_text)

# Function to map to vectorization layer
def vectorize_text(text, label):
    text = tf.expand_dims(text["sentence"], -1)
    return bin_vectorize_layer(text), label

train_ds = train_ds.map(vectorize_text)
val_ds = val_ds.map(vectorize_text)
test_ds = test_ds.map(vectorize_text)

train_ds = train_ds.map(lambda x, y: (x, tf.one_hot(y, 4)))
val_ds = val_ds.map(lambda x, y: (x, tf.one_hot(y, 4)))
test_ds = test_ds.map(lambda x, y: (x, tf.one_hot(y, 4)))

model = keras.Sequential([
    layers.Input((VOCAB, )),
    layers.Dense(150),
    layers.Dense(4, activation="softmax")
])

model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy", metrics.Precision(name="precision"), metrics.Recall(name="recall")]
)

history = model.fit(
    train_ds,
    epochs=20,
    validation_data=val_ds,
    callbacks=[lr_decrease, early_stopping]
)

# Creates the training results
train_results = [
    name,
    str(model_index),
    history.history["accuracy"][-1],
    history.history["precision"][-1],
    history.history["recall"][-1],
    calculate_f1_score(history.history["precision"][-1], history.history["recall"][-1])
]

# Stores the training results
write_results_on_file(train_name, train_results)

# Creates the validation results
val_results = [
    name,
    str(model_index),
    history.history["val_accuracy"][-1],
    history.history["val_precision"][-1],
    history.history["val_recall"][-1],
    calculate_f1_score(history.history["val_precision"][-1], history.history["val_recall"][-1])
]

# Stores the validation results
write_results_on_file(val_name, val_results)

# Evaluates the model
results = model.evaluate(
    test_ds
)

# Appends model to list
results.insert(0, str(model_index))
results.insert(0, name)

# Removes loss from results
results.pop(2)

# Adds F1 score to test
results.append(calculate_f1_score(results[3], results[4]))

# Writes results
write_results_on_file(test_name, results)



Epoch 1/20
1417/1417 [==============================] - 6s 4ms/step - loss: 1.2833 - accuracy: 0.4331 - precision: 0.6143 - recall: 0.0543 - val_loss: 1.1468 - val_accuracy: 0.5152 - val_precision: 0.7255 - val_recall: 0.1895
Epoch 2/20
1417/1417 [==============================] - 5s 4ms/step - loss: 1.1066 - accuracy: 0.5251 - precision: 0.7348 - recall: 0.2273 - val_loss: 1.0991 - val_accuracy: 0.5333 - val_precision: 0.6877 - val_recall: 0.2739
Epoch 3/20
1417/1417 [==============================] - 5s 3ms/step - loss: 1.0282 - accuracy: 0.5630 - precision: 0.7352 - recall: 0.3132 - val_loss: 1.0914 - val_accuracy: 0.5345 - val_precision: 0.6660 - val_recall: 0.3152
Epoch 4/20
1417/1417 [==============================] - 4s 3ms/step - loss: 0.9762 - accuracy: 0.5918 - precision: 0.7454 - recall: 0.3686 - val_loss: 1.0975 - val_accuracy: 0.5334 - val_precision: 0.6498 - val_recall: 0.3359

Epoch 00004: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [======

In [52]:
train_ds = tf.data.experimental.make_csv_dataset(
    "./data/simpsons/train_ds.csv",
    batch_size=32,
    column_names = ["sentence", "character"],
    label_name="character",
    num_epochs=1
)

val_ds = tf.data.experimental.make_csv_dataset(
    "./data/simpsons/val_ds.csv",
    batch_size=32,
    column_names = ["sentence", "character"],
    label_name="character",
    num_epochs=1
)

test_ds = tf.data.experimental.make_csv_dataset(
    "./data/simpsons/test_ds.csv",
    batch_size=32,
    column_names = ["sentence", "character"],
    label_name="character",
    num_epochs=1
)

# Sets the vocabulary size and sequence length
VOCAB = 10000
MAX_TOKENS=15

# Sets the vectorization
VECT = "tf-idf"

# Creates the LR decrease callback
lr_decrease = callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    patience=1,
    cooldown=3,
    factor=0.1,
    verbose=1
)

# Creates a callback for early stopping
early_stopping = callbacks.EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=0
)

In [53]:
# Sets the task name
task_name = "tfidf_vectorization"

name=task_name

# Defines the results filenames
train_name = task_name + "_" + "simpsons_train"
val_name = task_name + "_" + "simpsons_val"
test_name = task_name + "_" + "simpsons_test"

# Creates the results file
create_results_file(train_name)
create_results_file(val_name)
create_results_file(test_name)

# Maps to create train text dataset
train_text = train_ds.map(lambda x, y: x["sentence"])

# Creates the vectorization layer
bin_vectorize_layer = layers.experimental.preprocessing.TextVectorization(
    output_mode=VECT,
    max_tokens=VOCAB,
    pad_to_max_tokens=MAX_TOKENS
)

# Trains the vectorization layer
bin_vectorize_layer.adapt(train_text)

# Function to map to vectorization layer
def vectorize_text(text, label):
    text = tf.expand_dims(text["sentence"], -1)
    return bin_vectorize_layer(text), label

train_ds = train_ds.map(vectorize_text)
val_ds = val_ds.map(vectorize_text)
test_ds = test_ds.map(vectorize_text)

train_ds = train_ds.map(lambda x, y: (x, tf.one_hot(y, 4)))
val_ds = val_ds.map(lambda x, y: (x, tf.one_hot(y, 4)))
test_ds = test_ds.map(lambda x, y: (x, tf.one_hot(y, 4)))

model = keras.Sequential([
    layers.Input((VOCAB, )),
    layers.Dense(150),
    layers.Dense(4, activation="softmax")
])

model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy", metrics.Precision(name="precision"), metrics.Recall(name="recall")]
)

history = model.fit(
    train_ds,
    epochs=20,
    validation_data=val_ds,
    callbacks=[lr_decrease, early_stopping]
)

# Creates the training results
train_results = [
    name,
    str(model_index),
    history.history["accuracy"][-1],
    history.history["precision"][-1],
    history.history["recall"][-1],
    calculate_f1_score(history.history["precision"][-1], history.history["recall"][-1])
]

# Stores the training results
write_results_on_file(train_name, train_results)

# Creates the validation results
val_results = [
    name,
    str(model_index),
    history.history["val_accuracy"][-1],
    history.history["val_precision"][-1],
    history.history["val_recall"][-1],
    calculate_f1_score(history.history["val_precision"][-1], history.history["val_recall"][-1])
]

# Stores the validation results
write_results_on_file(val_name, val_results)

# Evaluates the model
results = model.evaluate(
    test_ds
)

# Appends model to list
results.insert(0, str(model_index))
results.insert(0, name)

# Removes loss from results
results.pop(2)

# Adds F1 score to test
results.append(calculate_f1_score(results[3], results[4]))

# Writes results
write_results_on_file(test_name, results)



Epoch 1/20
1417/1417 [==============================] - 10s 7ms/step - loss: 1.2501 - accuracy: 0.4413 - precision: 0.6229 - recall: 0.1254 - val_loss: 1.1177 - val_accuracy: 0.5254 - val_precision: 0.6738 - val_recall: 0.2838
Epoch 2/20
1417/1417 [==============================] - 9s 6ms/step - loss: 1.0073 - accuracy: 0.5753 - precision: 0.7309 - recall: 0.3439 - val_loss: 1.1365 - val_accuracy: 0.5218 - val_precision: 0.6325 - val_recall: 0.3412

Epoch 00002: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
304/304 [==============================] - 2s 5ms/step - loss: 1.1351 - accuracy: 0.5136 - precision: 0.6143 - recall: 0.3315


In [60]:
train_ds = tf.data.experimental.make_csv_dataset(
    "./data/friends/train_ds.csv",
    batch_size=32,
    column_names = ["sentence", "character"],
    label_name="character",
    num_epochs=1
)

val_ds = tf.data.experimental.make_csv_dataset(
    "./data/friends/val_ds.csv",
    batch_size=32,
    column_names = ["sentence", "character"],
    label_name="character",
    num_epochs=1
)

test_ds = tf.data.experimental.make_csv_dataset(
    "./data/friends/test_ds.csv",
    batch_size=32,
    column_names = ["sentence", "character"],
    label_name="character",
    num_epochs=1
)

# Sets the vocabulary size and sequence length
VOCAB = 15000
MAX_TOKENS=20

# Sets the vectorization
VECT = "binary"

# Creates the LR decrease callback
lr_decrease = callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    patience=1,
    cooldown=3,
    factor=0.1,
    verbose=1
)

# Creates a callback for early stopping
early_stopping = callbacks.EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=0
)

In [61]:
# Sets the task name
task_name = "binary_vectorization"

name=task_name

# Defines the results filenames
train_name = task_name + "_" + "friends_train"
val_name = task_name + "_" + "friends_val"
test_name = task_name + "_" + "friends_test"

# Creates the results file
create_results_file(train_name)
create_results_file(val_name)
create_results_file(test_name)

# Maps to create train text dataset
train_text = train_ds.map(lambda x, y: x["sentence"])

# Creates the vectorization layer
bin_vectorize_layer = layers.experimental.preprocessing.TextVectorization(
    output_mode=VECT,
    max_tokens=VOCAB,
    pad_to_max_tokens=MAX_TOKENS
)

# Trains the vectorization layer
bin_vectorize_layer.adapt(train_text)

# Function to map to vectorization layer
def vectorize_text(text, label):
    text = tf.expand_dims(text["sentence"], -1)
    return bin_vectorize_layer(text), label

train_ds = train_ds.map(vectorize_text)
val_ds = val_ds.map(vectorize_text)
test_ds = test_ds.map(vectorize_text)

train_ds = train_ds.map(lambda x, y: (x, tf.one_hot(y, 6)))
val_ds = val_ds.map(lambda x, y: (x, tf.one_hot(y, 6)))
test_ds = test_ds.map(lambda x, y: (x, tf.one_hot(y, 6)))

model = keras.Sequential([
    layers.Input((VOCAB, )),
    layers.Dense(75),
    layers.Dense(150),
    layers.Dense(6, activation="softmax")
])

model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy", metrics.Precision(name="precision"), metrics.Recall(name="recall")]
)

history = model.fit(
    train_ds,
    epochs=20,
    validation_data=val_ds,
    callbacks=[lr_decrease, early_stopping]
)

# Creates the training results
train_results = [
    name,
    str(model_index),
    history.history["accuracy"][-1],
    history.history["precision"][-1],
    history.history["recall"][-1],
    calculate_f1_score(history.history["precision"][-1], history.history["recall"][-1])
]

# Stores the training results
write_results_on_file(train_name, train_results)

# Creates the validation results
val_results = [
    name,
    str(model_index),
    history.history["val_accuracy"][-1],
    history.history["val_precision"][-1],
    history.history["val_recall"][-1],
    calculate_f1_score(history.history["val_precision"][-1], history.history["val_recall"][-1])
]

# Stores the validation results
write_results_on_file(val_name, val_results)

# Evaluates the model
results = model.evaluate(
    test_ds
)

# Appends model to list
results.insert(0, str(model_index))
results.insert(0, name)

# Removes loss from results
results.pop(2)

# Adds F1 score to test
results.append(calculate_f1_score(results[3], results[4]))

# Writes results
write_results_on_file(test_name, results)


Epoch 1/20
1017/1017 [==============================] - 5s 4ms/step - loss: 1.7781 - accuracy: 0.2142 - precision: 0.0760 - recall: 4.8171e-06 - val_loss: 1.7259 - val_accuracy: 0.2670 - val_precision: 0.9091 - val_recall: 0.0014
Epoch 2/20
1017/1017 [==============================] - 4s 4ms/step - loss: 1.6646 - accuracy: 0.3266 - precision: 0.8042 - recall: 0.0082 - val_loss: 1.6796 - val_accuracy: 0.2934 - val_precision: 0.5929 - val_recall: 0.0311
Epoch 3/20
1017/1017 [==============================] - 3s 3ms/step - loss: 1.5304 - accuracy: 0.3947 - precision: 0.7875 - recall: 0.0716 - val_loss: 1.6801 - val_accuracy: 0.2997 - val_precision: 0.5274 - val_recall: 0.0677

Epoch 00003: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
218/218 [==============================] - 1s 3ms/step - loss: 1.6759 - accuracy: 0.3024 - precision: 0.5610 - recall: 0.0673


In [62]:
train_ds = tf.data.experimental.make_csv_dataset(
    "./data/friends/train_ds.csv",
    batch_size=32,
    column_names = ["sentence", "character"],
    label_name="character",
    num_epochs=1
)

val_ds = tf.data.experimental.make_csv_dataset(
    "./data/friends/val_ds.csv",
    batch_size=32,
    column_names = ["sentence", "character"],
    label_name="character",
    num_epochs=1
)

test_ds = tf.data.experimental.make_csv_dataset(
    "./data/friends/test_ds.csv",
    batch_size=32,
    column_names = ["sentence", "character"],
    label_name="character",
    num_epochs=1
)

# Sets the vocabulary size and sequence length
VOCAB = 15000
MAX_TOKENS=20

# Sets the vectorization
VECT = "tf-idf"

# Creates the LR decrease callback
lr_decrease = callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    patience=1,
    cooldown=3,
    factor=0.1,
    verbose=1
)

# Creates a callback for early stopping
early_stopping = callbacks.EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=0
)

In [63]:
# Sets the task name
task_name = "tfidf_vectorization"

name=task_name

# Defines the results filenames
train_name = task_name + "_" + "friends_train"
val_name = task_name + "_" + "friends_val"
test_name = task_name + "_" + "friends_test"

# Creates the results file
create_results_file(train_name)
create_results_file(val_name)
create_results_file(test_name)

# Maps to create train text dataset
train_text = train_ds.map(lambda x, y: x["sentence"])

# Creates the vectorization layer
bin_vectorize_layer = layers.experimental.preprocessing.TextVectorization(
    output_mode=VECT,
    max_tokens=VOCAB,
    pad_to_max_tokens=MAX_TOKENS
)

# Trains the vectorization layer
bin_vectorize_layer.adapt(train_text)

# Function to map to vectorization layer
def vectorize_text(text, label):
    text = tf.expand_dims(text["sentence"], -1)
    return bin_vectorize_layer(text), label

train_ds = train_ds.map(vectorize_text)
val_ds = val_ds.map(vectorize_text)
test_ds = test_ds.map(vectorize_text)

train_ds = train_ds.map(lambda x, y: (x, tf.one_hot(y, 6)))
val_ds = val_ds.map(lambda x, y: (x, tf.one_hot(y, 6)))
test_ds = test_ds.map(lambda x, y: (x, tf.one_hot(y, 6)))

model = keras.Sequential([
    layers.Input((VOCAB, )),
    layers.Dense(75),
    layers.Dense(150),
    layers.Dense(6, activation="softmax")
])

model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy", metrics.Precision(name="precision"), metrics.Recall(name="recall")]
)

history = model.fit(
    train_ds,
    epochs=20,
    validation_data=val_ds,
    callbacks=[lr_decrease, early_stopping]
)

# Creates the training results
train_results = [
    name,
    str(model_index),
    history.history["accuracy"][-1],
    history.history["precision"][-1],
    history.history["recall"][-1],
    calculate_f1_score(history.history["precision"][-1], history.history["recall"][-1])
]

# Stores the training results
write_results_on_file(train_name, train_results)

# Creates the validation results
val_results = [
    name,
    str(model_index),
    history.history["val_accuracy"][-1],
    history.history["val_precision"][-1],
    history.history["val_recall"][-1],
    calculate_f1_score(history.history["val_precision"][-1], history.history["val_recall"][-1])
]

# Stores the validation results
write_results_on_file(val_name, val_results)

# Evaluates the model
results = model.evaluate(
    test_ds
)

# Appends model to list
results.insert(0, str(model_index))
results.insert(0, name)

# Removes loss from results
results.pop(2)

# Adds F1 score to test
results.append(calculate_f1_score(results[3], results[4]))

# Writes results
write_results_on_file(test_name, results)


Epoch 1/20
1017/1017 [==============================] - 11s 10ms/step - loss: 1.7724 - accuracy: 0.2156 - precision: 0.4503 - recall: 0.0015 - val_loss: 1.6924 - val_accuracy: 0.2794 - val_precision: 0.6624 - val_recall: 0.0149
Epoch 2/20
1017/1017 [==============================] - 11s 10ms/step - loss: 1.5527 - accuracy: 0.3860 - precision: 0.8031 - recall: 0.0590 - val_loss: 1.7001 - val_accuracy: 0.2936 - val_precision: 0.5009 - val_recall: 0.0792

Epoch 00002: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
218/218 [==============================] - 2s 7ms/step - loss: 1.6953 - accuracy: 0.2985 - precision: 0.5310 - recall: 0.0775
